<a href="https://colab.research.google.com/github/nsydn/isu_kis_okulu/blob/main/supply_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Supply Network**

<img src="https://github.com/nsydn/kis_okulu/blob/main/image3.png?raw=1" alt="Drawing" style="width: 600px;"/>

### **Problem tanımı** (Problem Definition)

Uluslararası bir üretim şirketinin Lojistik Ağı Yöneticisi olarak, taşıma maliyetlerinde son zamanlarda görülen artışı ve gelecekteki talep tahminlerini göz önünde bulundurarak şirketin lojistik ağını önümüzdeki 5 yıl için yeniden tasarlamak istiyorsunuz.

#### **Ağ yapısı** (Network Structure)
* Brezilya, ABD, Hindistan, Japonya, Almanya olmak üzere 5 ülkede üretim ve satış yapıyoruz
* Her ülkede 2 tip üretim tesisinden biri kurulabilir: düşük ya da yüksek kapasite
* Taşıma maliyetleri (USD/konteyner)
* Piyasa talebi (adet/ay)

#### **Maliyetler** (Costs)
* Sabit maliyetler (fixed costs) (makina/ekipman, kira, vb.): $f$
* Değişken maliyetler (variable costs): $v=v_1 + v_2$
  * Üretim değişken maliyetleri (manufacturing variable costs) (üretim hattı operatörleri, hammadde, vb.): $v_1$
  * Taşıma değişken maliyetleri (freight variable costs) (1 konteyner = 1000 adet): $v_2$

### **Bilgisayar Modeli** (Computer Model)

In [1]:
%pip install pulp
import pandas as pd
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
# upload data
from google.colab import files
uploaded = files.upload()

Saving data.xlsx to data.xlsx


In [83]:
# read data
fixed_costs = pd.read_excel('data.xlsx', 'fixed_costs',index_col = 0)
manuf_costs = pd.read_excel('data.xlsx', 'manuf_costs', index_col = 0) #per_item
freight_costs = pd.read_excel('data.xlsx', 'freight_costs', index_col = 0)/1000 #per_container
var_costs = manuf_costs + freight_costs #per_item
capacities = pd.read_excel('data.xlsx', 'capacities', index_col = 0) #per_month
demands = pd.read_excel('data.xlsx', 'demands', index_col = 0) #per_month

In [60]:
# introduce locs and facility types
locs = ['US', 'Germany', 'Japan', 'Brazil', 'India']
types = ['low', 'high']

In [61]:
# initiate the model
my_model = LpProblem('network',LpMinimize)

#### **Karar değişkenleri** (Decision variables)
* $x_{i,j}$: Lokasyon $i$'den lokasyon $j$'ye transfer miktarı ($i,j\in$ `locs`)
* $y_{i,k}$: Lokasyon $i$'de $k$ kapasiteli tesis kurulacaksa 1, yoksa 0 ($i\in$ `locs`, $k\in$ `types`)

In [62]:
x = LpVariable.dicts('x', [(i,j) for i in locs for j in locs], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locs for k in types], cat='Binary')

#### **Amaç fonksiyonu** (Objective function)
Toplam tesis kurma ve dağıtım maliyetlerini minimize edecek bir dağıtım ağı kurmak istiyoruz:

$$\sum_{i} \sum_{k} f_{i,t} y_{i,k} + \sum_{i} \sum_{j} v_{i,j} x_{i,j}$$

In [63]:
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locs for k in types]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locs for j in locs]))

#### **Kısıtlar** (Constraints)
* Her bir $i$ lokasyonunda yalnızca bir tip tesis kurabiliriz (`low` ya da `high`):
$$\sum_{k} y_{i,k} \leq 1\quad ∀i$$
* Herbir $i$ lokasyonundan çıkan toplam transferin o lokasyondaki kapasiteyi aşmadığından emin olmalıyız: $$\sum_{j} x_{i,j} \leq \sum_{k} c_i y_k \quad ∀i$$
* Herbir $j$ lokasyonuna gelen toplam transferin o lokasyondaki talebi tam olarak karşıladığından emin olmalıyız: $$\sum_{i} x_{i,j} = d_j \quad ∀j$$

In [64]:
for i in locs:
  my_model += lpSum([y[i,k] for k in types]) <= 1
for i in locs:
  my_model += lpSum([x[i,j] for j in locs]) <= lpSum([capacities.loc[i,k] * y[i,k] for k in types])
for j in locs:
  my_model += lpSum([x[i,j] for i in locs]) == demands.loc[j,'demand']

#### **Modelin çözümü** (Solving the model)

In [65]:
my_model.solve()
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 67,768,475 ($/month)
Solution status: Optimal


#### **Sonuçlar** (Results)
* Hangi lokasyona hangi kapasitede bir tesis açıyoruz?
* Hangi lokasyonlar arası ne kadar ürün transfer ediyoruz?

In [66]:
plants = pd.DataFrame(index = locs, columns = types)
for i in locs:
  for k in types:
    plants.loc[i,k] = y[i,k].varValue
plants

,low,high
US,0.0,0.0
Germany,1.0,0.0
Japan,0.0,1.0
Brazil,0.0,1.0
India,0.0,1.0


In [67]:
transfers = pd.DataFrame(index = locs, columns = locs)
for i in locs:
  for j in locs:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
US,0.0,0.0,0.0,0.0,0.0
Germany,105000.0,90000.0,200000.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,1355000.0,0.0,0.0,145000.0,0.0
India,1340000.0,0.0,0.0,0.0,160000.0


#### **Senaryo A:** Üretimi işgücü maliyeti düşük olan bölgeye kaydırma
Hindistan'daki yüksek kapasiteli fabrikanın kapasitesini 2 katına çıkarırsak ne olur? (Sabit maliyetin de 2 katına çıktığını varsayalım.)

In [84]:
capacities.loc['India','high'] *= 2
fixed_costs.loc['India','high'] *= 2

In [85]:
my_model = LpProblem('network',LpMinimize)
x = LpVariable.dicts('x', [(i,j) for i in locs for j in locs], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locs for k in types], cat='Binary')
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locs for k in types]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locs for j in locs]))
for i in locs:
  my_model += lpSum([y[i,k] for k in types]) <= 1
for i in locs:
  my_model += lpSum([x[i,j] for j in locs]) <= lpSum([capacities.loc[i,k]*y[i,k] for k in types])
for j in locs:
  my_model += lpSum([x[i,j] for i in locs]) == demands.loc[j,'demand']
my_model.solve()
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 58,108,000 ($/month)
Solution status: Optimal


In [86]:
plants = pd.DataFrame(index = locs, columns = types)
for i in locs:
  for k in types:
    plants.loc[i,k] = y[i,k].varValue
plants

,low,high
US,0.0,0.0
Germany,0.0,0.0
Japan,0.0,1.0
Brazil,1.0,0.0
India,0.0,1.0


In [87]:
transfers = pd.DataFrame(index = locs, columns = locs)
for i in locs:
  for j in locs:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
US,0.0,0.0,0.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,250000.0,0.0,0.0,145000.0,0.0
India,2550000.0,90000.0,200000.0,0.0,160000.0


#### **Senaryo B:** Konteynır kısıtı nedeniyle artan taşıma maliyetleri
Konteynır kısıtı nedeniyle taşıma maliyetleri 5 katına çıkarsa ne olur?

In [88]:
# reset scenario 1 values
fixed_costs = pd.read_excel('data.xlsx', 'fixed_costs',index_col = 0)
capacities = pd.read_excel('data.xlsx', 'capacities', index_col = 0) #per_month

In [89]:
freight_costs *= 5
var_costs = manuf_costs + freight_costs #per_item

In [90]:
my_model = LpProblem('network',LpMinimize)
x = LpVariable.dicts('x', [(i,j) for i in locs for j in locs], lowBound=0, upBound=None, cat='Continuous')
y = LpVariable.dicts('y', [(i,k) for i in locs for k in types], cat='Binary')
my_model += (lpSum([fixed_costs.loc[i,k] * y[i,k] for i in locs for k in types]) 
          + lpSum([var_costs.loc[i,j] * x[i,j] for i in locs for j in locs]))
for i in locs:
  my_model += lpSum([y[i,k] for k in types]) <= 1
for i in locs:
  my_model += lpSum([x[i,j] for j in locs]) <= lpSum([capacities.loc[i,k]*y[i,k] for k in types])
for j in locs:
  my_model += lpSum([x[i,j] for i in locs]) == demands.loc[j,'demand']
my_model.solve()
print('Total cost = {:,} ($/month)'.format(int(value(my_model.objective))))
print("Solution status: {}".format(LpStatus[my_model.status]))

Total cost = 85,785,000 ($/month)
Solution status: Optimal


In [91]:
plants = pd.DataFrame(index = locs, columns = types)
for i in locs:
  for k in types:
    plants.loc[i,k] = y[i,k].varValue
plants

,low,high
US,0.0,1.0
Germany,0.0,0.0
Japan,0.0,1.0
Brazil,1.0,0.0
India,0.0,1.0


In [92]:
transfers = pd.DataFrame(index = locs, columns = locs)
for i in locs:
  for j in locs:
    transfers.loc[i,j] = x[i,j].varValue
transfers  

,US,Germany,Japan,Brazil,India
US,1300000.0,0.0,200000.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0
Japan,0.0,0.0,1500000.0,0.0,0.0
Brazil,250000.0,0.0,0.0,145000.0,0.0
India,1250000.0,90000.0,0.0,0.0,160000.0
